In [25]:
dtypes = {
    'comment_text':'str',
    'comment_id':'str',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str'
}
#,
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count']
#comment_author
content_features = ['comments']
labels = ['rude','figurative','offensive','dirty']


reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [26]:
import re
import string
import pythainlp
from pythainlp import word_tokenize
from pythainlp.corpus import wordnet
from nltk.stem.porter import PorterStemmer
from nltk.corpus import words
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import xlrd  
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

p_stemmer = PorterStemmer()

def split_word(text):
    
    tokens = word_tokenize(text,engine='newmm')

    # หารากศัพท์ภาษาไทย และภาษาอังกฤษ
    # English
    tokens = [p_stemmer.stem(i) for i in tokens]
    
    # Thai
    tokens_temp=[]
    for i in tokens:
        w_syn = wordnet.synsets(i)
        if (len(w_syn)>0) and (len(w_syn[0].lemma_names('tha'))>0):
            tokens_temp.append(w_syn[0].lemma_names('tha')[0])
        else:
            tokens_temp.append(i)
    
    tokens = tokens_temp
    
    # ลบตัวเลข
    tokens = [i for i in tokens if not i.isnumeric()]
    
    # ลบช่องว่าง
    tokens = [i for i in tokens if not ' ' in i]

    return tokens


# Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/df_labeled_temp.csv', dtype=dtypes)
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non.csv', dtype=dtypes)
Corpus.rename(columns={'comment_text': 'comments'}, inplace=True)


#unlabel need to be changed to the real one
#Unlabeled_Corpus = pd.read_excel('pimrypiefc_comment.xlsx')
Unlabeled_Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/unlabeled_data_temp1.csv', dtype=dtypes)
Unlabeled_Corpus.rename(columns={'comment_text': 'comments'}, inplace=True)

#remove duplicate
label_id_set = set(Corpus['comment_id'])
unlabel_id_set = set(Unlabeled_Corpus['comment_id'])
duplicate_comment_id_set = label_id_set.intersection(unlabel_id_set)
Unlabeled_Corpus = Unlabeled_Corpus.loc[~Unlabeled_Corpus['comment_id'].isin(duplicate_comment_id_set)]
Unlabeled_Corpus.reset_index(inplace=True)



In [27]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)
Unlabeled_Corpus[reaction_columns] = Unlabeled_Corpus[reaction_columns].replace(reaction_map)


In [28]:
Train_X, Test_X, Train_Y, Test_Y = train_test_split(
    Corpus[content_features + context_features], 
    Corpus[labels], test_size=0.2, random_state=1000);

#Tokenizaiton
Tfidf_vect = TfidfVectorizer(tokenizer=split_word, min_df=2, max_features=10000, token_pattern=None)

#fit only train, unlabel
Tfidf_vect.fit(pd.concat([Train_X['comments'],Unlabeled_Corpus['comments']]))

#transform train, test, unlabel
Train_X_Tfidf = Tfidf_vect.transform(Train_X['comments'])
Test_X_Tfidf = Tfidf_vect.transform(Test_X['comments'])
Unlabeled_X_Tfidf = Tfidf_vect.transform(Unlabeled_Corpus['comments'])


In [29]:
def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    
def createVotingEvaluation(file):
    f = open(file, mode='w')
    f.write("data_type,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    
def createCombinedEvaluation(file):
    f = open(file, mode='w')
    f.write("data_type,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()
    
def createWeightAvgEvaluation(file):
    f = open(file, mode='w')
    f.write("data_type,alpha,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()

In [30]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [31]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def report_performance_voting(file,modelType, predictions_class_test_c, type_class ,predict_test_prob):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    f = open(file, mode='a')
    f.write(data_type_test+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.close()
    
def report_performance_combined(file,modelType,th, predictions_class_test_c, type_class ,predict_test_prob):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    type_class = str(type_class)
    threshold = str(th)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    f = open(file, mode='a')
    f.write(data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.close()

In [32]:
def report_performance_weightavg(file, modelType, predictions_class_test_c, type_class, alpha, threshold, predict_test_prob):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    alpha = str(alpha)
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    f = open(file, mode='a')
    f.write(data_type_test+","+alpha+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.close()

In [67]:
from scipy.sparse import hstack, csr_matrix, vstack
train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
unlabeled_feat_context = Unlabeled_Corpus[context_features].values

train_feat_content = Train_X_Tfidf
test_feat_content = Test_X_Tfidf
unlabeled_feat_content = Unlabeled_X_Tfidf

In [34]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
    
def train_and_label_context(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType, l, u):
    model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    model.fit(train_X, train_y)
    
    # eval
    predictions_test = model.predict_proba(test_X)
    predictions_train = model.predict_proba(train_X)
    
    maxfscore = -1.0;
    threshold = 0;
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')
        
        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])
        
        precision,recall,fscore,support=score(test_y,predictions_class_Test)
        fscore_test = float(format(fscore[1]))
        if(fscore_test > maxfscore):
            maxfscore = fscore_test
            index = time-1
            threshold = th
        
    # add new data
    
    predictions_unlabeled = model.predict_proba(u_X)
    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > u).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < l).reshape(-1)
    return certain_positives, certain_negatives, index, threshold, maxfscore

In [35]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
    
def train_and_label_content(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType, l, u):
    if(type_class == 'dirty'):
        model = LinearSVC()
    model.fit(train_X, train_y)
    
    # eval
    if(type_class != 'dirty'):
        predictions_test = model.predict_proba(test_X)
        predictions_train = model.predict_proba(train_X)
        predictions_unlabeled = model.predict_proba(u_X)
    
    elif(type_class == 'dirty'):
        predictions_test = model._predict_proba_lr(test_X)
        predictions_train = model._predict_proba_lr(train_X)
        predictions_unlabeled = model._predict_proba_lr(u_X)
    
    maxfscore = -1.0;
    threshold = 0;
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')
        
        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])
        
        precision,recall,fscore,support=score(test_y,predictions_class_Test)
        fscore_test = float(format(fscore[1]))

        if(fscore_test > maxfscore):
            maxfscore = fscore_test
            index = time-1
            threshold = th
    # add new data
    
    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > u).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < l).reshape(-1)
    return certain_positives, certain_negatives, index, threshold, maxfscore

In [36]:
from sklearn import tree
def train_and_label_combined(train_X, train_y, test_X, test_y, type_class, file):
    if(type_class == 'dirty'):
        model = tree.DecisionTreeClassifier()
    model.fit(train_X, train_y)
    
    # eval
    predictions_test = model.predict_proba(test_X)
    
    print(predictions_test)
    maxfscore = -1.0;
    threshold = 0;
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')
        
        report_performance_combined(file,"combined",th,predictions_class_Test, type_class, predictions_test[:,1])

        precision,recall,fscore,support=score(test_y,predictions_class_Test)
        fscore_test = float(format(fscore[1]))

        if(fscore_test > maxfscore):
            maxfscore = fscore_test
            threshold = th
    # add new data
    
    return threshold

In [37]:
def voting(train_x_content, train_y_content,content_th, train_x_context, train_y_context,context_th, 
           combined_train_x, combined_train_y,combined_th,pre_x_content, pre_y_content, pre_content_th, 
           pre_x_context, pre_y_context, pre_context_th, 
           test_content_X, test_context_X, test_combined_X,type_class):
    
    predictions_class_Test = 0;
    predictions_class_Train = 0;
    model_context = RandomForestClassifier(n_estimators=20, n_jobs=20)
    model_pre_context = RandomForestClassifier(n_estimators=20, n_jobs=20)
    
    if(type_class == 'dirty'):
        model_content = LinearSVC()
        model_combined = tree.DecisionTreeClassifier()
        model_pre_content = LinearSVC()
        
    
    model_content.fit(train_x_content, train_y_content)
    model_context.fit(train_x_context, train_y_context)
    model_combined.fit(combined_train_x, combined_train_y)
    model_pre_content.fit(pre_x_content, pre_y_content)
    model_pre_context.fit(pre_x_context, pre_y_context)
    
    if(type_class == 'dirty'):
        predictions_test_content = model_content._predict_proba_lr(test_content_X)
        predictions_class_Test = predictions_class_Test + (predictions_test_content[:,1] >= content_th).astype('int')
    
        predictions_test_pre_content = model_pre_content._predict_proba_lr(test_content_X)
        predictions_class_Test = predictions_class_Test + (predictions_test_pre_content[:,1] >= pre_content_th).astype('int')
    
    predictions_test_context = model_context.predict_proba(test_context_X)
    predictions_class_Test = predictions_class_Test + (predictions_test_context[:,1] >= context_th).astype('int')
    
    predictions_test_pre_context = model_pre_context.predict_proba(test_context_X)
    predictions_class_Test = predictions_class_Test + (predictions_test_pre_context[:,1] >= pre_context_th).astype('int')
    
    predictions_test_combined = model_combined.predict_proba(test_combined_X)
    predictions_class_Test = predictions_class_Test + (predictions_test_combined[:,1] >= combined_th).astype('int')
    
    predictions_class_Test[predictions_class_Test < 3] = 0
    predictions_class_Test[predictions_class_Test >= 3] = 1
    
    predictions_prob_test = (predictions_test_content + predictions_test_context +  predictions_test_pre_content + predictions_test_pre_context + predictions_test_combined)/5

    report_performance_voting(file,"voting",predictions_class_Test, type_class, predictions_prob_test[:,1])
        

In [38]:
def predict_class(list_predict_prob,threshold):
    list_predict_class = [];
    for i in range (len(list_predict_prob)):
        if(list_predict_prob[i][1] > threshold):
            list_predict_class.append(1)
        else :
            list_predict_class.append(0)
    return list_predict_class

def weightAvg(file, train_x_content, train_y_content, train_x_context, train_y_context,
             test_content_X, test_context_X,type_class):
    for alpha in numpy.arange(0.0, 1.1, 0.1):
        if(type_class == 'dirty'):
            model_content = LinearSVC()
        
        model_context = RandomForestClassifier(n_estimators=20, n_jobs=20)
        model_content.fit(train_x_content, train_y_content)
        model_context.fit(train_x_context, train_y_context)
        
        if(type_class == 'dirty'):
            predictions_test_content = model_content._predict_proba_lr(test_content_X)
            
        predictions_test_context = model_context.predict_proba(test_context_X)
        
        for threshold in numpy.arange(0.0, 1.1, 0.1):
            predictions_Test = (predictions_test_content*alpha)+(predictions_test_context*(1-alpha))
            predictions_class_Test = predict_class(predictions_Test,threshold)
            
            report_performance_weightavg(file, "weightavg", predictions_class_Test, type_class,alpha, threshold, predictions_Test[:,1])

In [68]:
file = 'test_co.csv'
createGeneralEvaluation(file)

In [69]:
from scipy.sparse import hstack, csr_matrix, vstack

#label and window will be changed depends on what class is experimented
label = 'dirty'
if(label == 'dirty'):
    low_win_context = 0.05
    up_win_context = 0.7
    low_win_content = 0.05
    up_win_content = 0.7

_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y
context_train_y = _train_y

content_certain_set = set()
content_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
content_certain_labels[label] = -1

context_certain_set = set()
context_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
context_certain_labels[label] = -1

co_training_content_X = [];
co_training_context_X = [];
content = [];

co_training_content_Y = [];
co_training_context_Y = [];
context = [];



for i in range(1,11):
    time = i
    co_training_content_X.append(train_feat_content)
    co_training_content_Y.append(content_train_y)
    co_training_context_X.append(train_feat_context)
    co_training_context_Y.append(context_train_y)
    # certain data from content
    _cp, _cn, c_iteration, c_th, c_fscore = train_and_label_content(
        train_feat_content, content_train_y, 
        test_feat_content, _test_y,
        unlabeled_feat_content, file, label, time, "content", low_win_content, up_win_content)
    content.append([c_iteration, c_th, c_fscore])
    
    # certain data from context
    _xp, _xn, x_iteration, x_th, x_fscore = train_and_label_context(
        train_feat_context, context_train_y, 
        test_feat_context, _test_y,
        unlabeled_feat_context, file, label, time, "context", low_win_context, up_win_context)
    context.append([x_iteration, x_th, x_fscore])
    
    # Update certain data from content to context model
    content_certain_set.update(list(_cp))
    content_certain_set.update(list(_cn))
    content_certain_labels.loc[_cp, label] = 1
    content_certain_labels.loc[_cn, label] = 0

    _ccl = list(content_certain_set)
#     start = 5000*(time-1)
#     end = 5000*(time)
#     _ccl = _ccl[start:end]
    _cc_X = unlabeled_feat_context[_ccl]
    _cc_y = content_certain_labels.loc[_ccl, label]

    train_feat_context = vstack([train_feat_context, _cc_X])
    context_train_y = np.concatenate([context_train_y, _cc_y.values], axis=0)
    
    print('current amount of data in context model'+str(train_feat_context.shape))
    
    # Update certain data from context to content model
    context_certain_set.update(list(_xp))
    context_certain_set.update(list(_xn))
    context_certain_labels.loc[_xp, label] = 1
    context_certain_labels.loc[_xn, label] = 0

    _xxl = list(context_certain_set)
#     _xxl = _xxl[start:end]
    _xx_X = unlabeled_feat_content[_xxl]
    _xx_y = context_certain_labels.loc[_xxl, label]

    train_feat_content = vstack([train_feat_content, _xx_X])
    content_train_y = np.concatenate([content_train_y, _xx_y.values], axis=0)

    print('current amount of data in content model'+str(train_feat_content.shape))

Test data of content
dirty Accuracy score: 3.769950467804073
Confusion Matrix: 
 [[  12 3497]
 [   0  125]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.01      3509
         1.0       0.03      1.00      0.07       125

    accuracy                           0.04      3634
   macro avg       0.52      0.50      0.04      3634
weighted avg       0.97      0.04      0.01      3634

Area under the curve ROC:  0.9631667141635794
Area under the curve PR:  0.682090439222957
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 3.577571379428965
Confusion Matrix: 
 [[   50 14015]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.01     14065
         1.0       0.03      1.00      0.06       470

    accuracy                           0.04     14535
   macro avg       0.52      0.50      0.03     14535
weighted avg   

PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       1.00      0.06      0.11       125

    accuracy                           0.97      3634
   macro avg       0.98      0.53      0.54      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC:  0.9631667141635794
Area under the curve PR:  0.682090439222957
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 96.83522531819746
Confusion Matrix: 
 [[14065     0]
 [  460    10]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     14065
         1.0       1.00      0.02      0.04       470

    accuracy                           0.97     14535
   macro avg       0.98      0.51      0.51     14535
weighted avg       0.97      0.97      0.95     14535

Area under the curve ROC:  0.9974997541808169
Area under the curve P

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

Test data of context
dirty Accuracy score: 89.76334617501377
Confusion Matrix: 
 [[3152  357]
 [  15  110]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.90      0.94      3509
         1.0       0.24      0.88      0.37       125

    accuracy                           0.90      3634
   macro avg       0.62      0.89      0.66      3634
weighted avg       0.97      0.90      0.92      3634

Area under the curve ROC:  0.9256483328583642
Area under the curve PR:  0.49596687461179456
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 95.74819401444789
Confusion Matrix: 
 [[13447   618]
 [    0   470]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.96      0.98     14065
         1.0       0.43      1.00      0.60       470

    accuracy                           0.96     14535
   macro avg       0.72      0.98      0.79     14535
weighted avg 

Confusion Matrix: 
 [[3509    0]
 [ 114   11]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       1.00      0.09      0.16       125

    accuracy                           0.97      3634
   macro avg       0.98      0.54      0.57      3634
weighted avg       0.97      0.97      0.96      3634

Area under the curve ROC:  0.9256483328583642
Area under the curve PR:  0.49596687461179456
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 97.55073959408324
Confusion Matrix: 
 [[14065     0]
 [  356   114]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.98      1.00      0.99     14065
         1.0       1.00      0.24      0.39       470

    accuracy                           0.98     14535
   macro avg       0.99      0.62      0.69     14535
weighted avg       0.98      0.98      0.97     14535

Area under the curv

...

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

PRF: 
               precision    recall  f1-score   support

         0.0       0.31      1.00      0.47     32373
         1.0       0.00      0.00      0.00     72039

    accuracy                           0.31    104412
   macro avg       0.16      0.50      0.24    104412
weighted avg       0.10      0.31      0.15    104412

Area under the curve ROC:  0.8936269421984919
Area under the curve PR:  0.9457153463795651
___________________________________________________________________________________________________________



/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test data of context
dirty Accuracy score: 85.27793065492571
Confusion Matrix: 
 [[2984  525]
 [  10  115]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.85      0.92      3509
         1.0       0.18      0.92      0.30       125

    accuracy                           0.85      3634
   macro avg       0.59      0.89      0.61      3634
weighted avg       0.97      0.85      0.90      3634

Area under the curve ROC:  0.9293758905671131
Area under the curve PR:  0.2925588023063464
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 96.96619696619697
Confusion Matrix: 
 [[13013  1182]
 [    0 24766]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.92      0.96     14195
         1.0       0.95      1.00      0.98     24766

    accuracy                           0.97     38961
   macro avg       0.98      0.96      0.97     38961
weighted avg  

Test data of context
dirty Accuracy score: 96.28508530544853
Confusion Matrix: 
 [[3491   18]
 [ 117    8]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      0.99      0.98      3509
         1.0       0.31      0.06      0.11       125

    accuracy                           0.96      3634
   macro avg       0.64      0.53      0.54      3634
weighted avg       0.94      0.96      0.95      3634

Area under the curve ROC:  0.9293758905671131
Area under the curve PR:  0.2925588023063464
-----------------------------------------------------------

Train data of context
dirty Accuracy score: 98.75516542183209
Confusion Matrix: 
 [[14195     0]
 [  485 24281]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98     14195
         1.0       1.00      0.98      0.99     24766

    accuracy                           0.99     38961
   macro avg       0.98      0.99      0.99     38961
weighted avg  

In [64]:
train_feat_content.shape

(613100, 10000)

In [65]:
train_feat_context.shape

(33670, 60)

In [17]:
# # error analysis co-training time10

# model = LinearSVC()
# model.fit(train_feat_content, content_train_y)

# predictions_test = model._predict_proba_lr(test_feat_content)
# predictions_class_Test = (predictions_test[:,1] >= content[9][1]).astype('int')

# f = open('error_analysis_dirty.csv', mode='w')
# f.write("comment_id,comment_text,actual_label, predicted_label")
# f.write("\n")
# f.close()
# f = open('error_analysis_dirty.csv', mode='a')
# for i,v in enumerate(Test_X.index):
#     t = Corpus.iloc[v]
#     f.write(t['comment_id']+","+t['comments']+","+str(t['dirty'])+","+str(predictions_class_Test[i]))
#     f.write("\n")
# f.close()


In [18]:
#get best content classifier after co-training
content_best_fscore= -1
for i in range(0,len(content)):
    if(content[i][2] > content_best_fscore):
        content_best_fscore = content[i][2]
        content_index = content[i][0]
        content_th = content[i][1]
        
#get best context classifier after co-training
context_best_fscore= -1
for i in range(0,len(context)):
    if(context[i][2] > context_best_fscore):
        context_best_fscore = context[i][2]
        context_index = context[i][0]
        context_th = context[i][1]

In [24]:
#voting
#classifier before co-training
pre_co_training_content_X =  co_training_content_X[0]
pre_co_training_content_Y =  co_training_content_Y[0]
pre_co_training_content_th = content[0][1]
pre_co_training_context_X = co_training_context_X[0]
pre_co_training_context_Y = co_training_context_Y[0]
pre_co_training_context_th = context[0][1]

#combined classifier
file = 'report_dirty_combined_addAllCertainData.csv'
createVotingEvaluation(file)
train_tfidf_content = Train_X_Tfidf
test_tfidf_content = Test_X_Tfidf

train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
    
train_feat = hstack([train_tfidf_content, train_feat_context])
test_feat = hstack([test_tfidf_content, test_feat_context])

b_th = train_and_label_combined(
    train_feat, Train_Y[label].values, 
    test_feat, Test_Y[label].values, label,file)

combined_train_x = train_feat
combined_train_y = Train_Y[label].values
combined_test_x = test_feat
combined_th = b_th

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]
Test data of combined
dirty Accuracy score: 97.93615850302697
Confusion Matrix: 
 [[3472   37]
 [  38   87]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.70      0.70      0.70       125

    accuracy                           0.98      3634
   macro avg       0.85      0.84      0.84      3634
weighted avg       0.98      0.98      0.98      3634

Area under the curve ROC:  0.842727842690225
Area under the curve PR:  0.7040348500719016
-----------------------------------------------------------

Test data of combined
dirty Accuracy score: 97.93615850302697
Confusion Matrix: 
 [[3472   37]
 [  38   87]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      3509
         1.0       0.70      0.70      0.70       125

    accuracy                           0.98      3634
   macro avg

In [ ]:
file = 'report_dirty_voting_addAllCertainData.csv'
createVotingEvaluation(file)

voting(co_training_content_X[content_index], co_training_content_Y[content_index], content_th, 
       co_training_context_X[context_index], co_training_context_Y[context_index], context_th,
       combined_train_x, combined_train_y,combined_th,
       pre_co_training_content_X, pre_co_training_content_Y, pre_co_training_content_th,
       pre_co_training_context_X, pre_co_training_context_Y, pre_co_training_context_th, 
       Test_X_Tfidf, Test_X[context_features].values, combined_test_x,label)

In [22]:
file = 'report_dirty_weightAvg_addAllCertainData.csv'
createWeightAvgEvaluation(file)

weightAvg(file, co_training_content_X[content_index], co_training_content_Y[content_index], 
          co_training_context_X[context_index], co_training_context_Y[context_index],
          Test_X_Tfidf, Test_X[context_features].values,label)

Test data of weightavg
dirty Accuracy score: 82.99394606494221
Confusion Matrix: 
 [[2895  614]
 [   4  121]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.83      0.90      3509
         1.0       0.16      0.97      0.28       125

    accuracy                           0.83      3634
   macro avg       0.58      0.90      0.59      3634
weighted avg       0.97      0.83      0.88      3634

Area under the curve ROC:  0.9432761470504417
Area under the curve PR:  0.47351668184827
-----------------------------------------------------------

Test data of weightavg
dirty Accuracy score: 90.94661529994497
Confusion Matrix: 
 [[3203  306]
 [  23  102]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.99      0.91      0.95      3509
         1.0       0.25      0.82      0.38       125

    accuracy                           0.91      3634
   macro avg       0.62      0.86      0.67      3634
weighted avg     

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test data of weightavg
dirty Accuracy score: 3.439735828288387
Confusion Matrix: 
 [[   0 3509]
 [   0  125]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3509
         1.0       0.03      1.00      0.07       125

    accuracy                           0.03      3634
   macro avg       0.02      0.50      0.03      3634
weighted avg       0.00      0.03      0.00      3634

Area under the curve ROC:  0.9625876318039327
Area under the curve PR:  0.536641608185709
-----------------------------------------------------------

Test data of weightavg
dirty Accuracy score: 88.74518436984039
Confusion Matrix: 
 [[3111  398]
 [  11  114]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.89      0.94      3509
         1.0       0.22      0.91      0.36       125

    accuracy                           0.89      3634
   macro avg       0.61      0.90      0.65      3634
weighted avg    

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       0.00      0.00      0.00       125

    accuracy                           0.97      3634
   macro avg       0.48      0.50      0.49      3634
weighted avg       0.93      0.97      0.95      3634

Area under the curve ROC:  0.9625876318039327
Area under the curve PR:  0.536641608185709
-----------------------------------------------------------

Test data of weightavg
dirty Accuracy score: 3.439735828288387
Confusion Matrix: 
 [[   0 3509]
 [   0  125]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3509
         1.0       0.03      1.00      0.07       125

    accuracy                           0.03      3634
   macro avg       0.02      0.50      0.03      3634
weighted avg       0.00      0.03      0.00      3634

Area under the curve ROC:  0.9637207181533203
Area under the curve PR: 

...

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test data of weightavg
dirty Accuracy score: 3.439735828288387
Confusion Matrix: 
 [[   0 3509]
 [   0  125]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      3509
         1.0       0.03      1.00      0.07       125

    accuracy                           0.03      3634
   macro avg       0.02      0.50      0.03      3634
weighted avg       0.00      0.03      0.00      3634

Area under the curve ROC:  0.9631667141635794
Area under the curve PR:  0.682090439222957
-----------------------------------------------------------

Test data of weightavg
dirty Accuracy score: 3.769950467804073
Confusion Matrix: 
 [[  12 3497]
 [   0  125]]
PRF: 
               precision    recall  f1-score   support

         0.0       1.00      0.00      0.01      3509
         1.0       0.03      1.00      0.07       125

    accuracy                           0.04      3634
   macro avg       0.52      0.50      0.04      3634
weighted avg    

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test data of weightavg
dirty Accuracy score: 96.58778205833792
Confusion Matrix: 
 [[3509    0]
 [ 124    1]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       1.00      0.01      0.02       125

    accuracy                           0.97      3634
   macro avg       0.98      0.50      0.50      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC:  0.9631667141635794
Area under the curve PR:  0.682090439222957
-----------------------------------------------------------

Test data of weightavg
dirty Accuracy score: 96.56026417171161
Confusion Matrix: 
 [[3509    0]
 [ 125    0]]
PRF: 
               precision    recall  f1-score   support

         0.0       0.97      1.00      0.98      3509
         1.0       0.00      0.00      0.00       125

    accuracy                           0.97      3634
   macro avg       0.48      0.50      0.49      3634
weighted avg    

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
